In [135]:
import numpy as np
import os
import json
from random import shuffle
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import Normalization
from sklearn.model_selection import train_test_split


In [136]:
# Load all the data from the json files
dir = r"src/dataFiveNotePatterns/"
positive_note_patterns = []
negative_note_patterns = []

for file in os.listdir(dir):
    with open(dir + file, "r", encoding='utf-8') as f:
        data = json.load(f)
        positive_note_patterns += data["five_notes_patterns"]
        negative_note_patterns += data["bad_patterns"]
        


print("Positive note patterns: ", len(positive_note_patterns))
print("Negative note patterns: ", len(negative_note_patterns))


Positive note patterns:  1905
Negative note patterns:  1656


In [137]:
## make sure the two lists have the same length
if(len(positive_note_patterns) > len(negative_note_patterns)):
    positive_note_patterns = positive_note_patterns[:len(negative_note_patterns)]
else:
    negative_note_patterns = negative_note_patterns[:len(positive_note_patterns)]

print("Positive note patterns: ", len(positive_note_patterns))
print("Negative note patterns: ", len(negative_note_patterns))
all_note_patterns = positive_note_patterns + negative_note_patterns

print(all_note_patterns)
# shuffle the data
shuffle(all_note_patterns)


Positive note patterns:  1656
Negative note patterns:  1656
[[[6, 2, 39360], [6, 2, 39840], [5, 4, 40320], [6, 2, 40800], [5, 5, 41280], [1]], [[6, 2, 41760], [5, 4, 42240], [6, 2, 42720], [5, 2, 43200], [6, 5, 43680], [1]], [[6, 4, 44160], [6, 5, 44640], [5, 2, 45120], [6, 5, 45600], [6, 4, 46080], [1]], [[6, 0, 46560], [6, 2, 47040], [6, 2, 47520], [5, 4, 48000], [6, 2, 48480], [1]], [[5, 5, 48960], [6, 2, 49440], [5, 4, 49920], [6, 2, 50400], [5, 2, 50880], [1]], [[6, 5, 51360], [6, 4, 51840], [6, 5, 52320], [5, 2, 52800], [6, 5, 53280], [1]], [[6, 4, 53760], [6, 0, 54240], [6, 2, 54720], [6, 2, 55200], [5, 4, 55680], [1]], [[6, 2, 56160], [5, 5, 56640], [6, 2, 57120], [5, 4, 57600], [6, 2, 58080], [1]], [[5, 2, 58560], [6, 5, 59040], [6, 4, 59520], [6, 5, 60000], [5, 2, 60480], [1]], [[6, 5, 60960], [6, 4, 61440], [6, 0, 61920], [6, 2, 62400], [6, 2, 62880], [1]], [[5, 4, 63360], [6, 2, 63840], [5, 5, 64320], [6, 2, 64800], [5, 4, 65280], [1]], [[3, 4, 84000], [3, 2, 84160], [3, 0,

In [138]:
all_fret_patterns = []
all_fret_patterns_labels = []
for pattern in all_note_patterns:
    all_fret_patterns.append([[string, fret] for string, fret, _ in pattern[:-1]])
    all_fret_patterns_labels.append(pattern[-1])

print(all_fret_patterns)
all_fret_patterns = np.array(all_fret_patterns)
all_fret_patterns_labels = np.array(all_fret_patterns_labels)

# Reshape the data to fit the model input
all_fret_patterns = all_fret_patterns.reshape((all_fret_patterns.shape[0], -1))

# split the data into training and testing
train_ratio = 0.8
X_train, X_test, y_train, y_test = train_test_split(all_fret_patterns, all_fret_patterns_labels, test_size=1-train_ratio)
for x, y in zip(X_train, y_train):
    print(x, y)

[[[3, 2], [1, 0], [2, 1], [3, 2], [2, 22]], [[4, 7], [4, 6], [5, 7], [5, 7], [6, 5]], [[4, 7], [4, 6], [5, 7], [5, 7], [3, 19]], [[4, 14], [4, 14], [4, 16], [3, 14], [3, 16]], [[4, 4], [4, 5], [4, 0], [4, 7], [4, 5]], [[5, 7], [5, 5], [6, 8], [6, 7], [6, 4]], [[6, 2], [6, 2], [6, 2], [6, 2], [6, 2]], [[2, 7], [2, 8], [2, 7], [3, 7], [2, 7]], [[2, 23], [5, 10], [6, 0], [6, 0], [6, 0]], [[2, 7], [3, 9], [3, 7], [4, 9], [3, 7]], [[0, 3], [3, 9], [3, 9], [3, 11], [3, 14]], [[2, 5], [3, 5], [4, 6], [5, 0], [4, 6]], [[1, 24], [3, 0], [2, 0], [1, 0], [2, 0]], [[5, 3], [6, 4], [5, 2], [6, 3], [6, 1]], [[2, 22], [6, 3], [6, 3], [6, 3], [6, 3]], [[1, 13], [2, 15], [2, 14], [1, 21], [1, 17]], [[4, 9], [4, 9], [4, 9], [2, 17], [4, 7]], [[2, 19], [6, 0], [6, 0], [6, 0], [6, 0]], [[5, 5], [5, 4], [5, 0], [5, 4], [4, 4]], [[3, 16], [1, 12], [2, 14], [0, 1], [1, 12]], [[5, 0], [5, 2], [5, 3], [4, 2], [3, 0]], [[5, 0], [4, 2], [3, 2], [2, 1], [5, 0]], [[2, 12], [2, 12], [2, 15], [1, 12], [1, 12]], [[3,

In [139]:
#noramlize the data
normalizer = Normalization()
normalizer.adapt(X_train)
X_train = normalizer(X_train)
X_test = normalizer(X_test)

In [140]:
#build the model
model = keras.Sequential([
    normalizer,
    keras.layers.Dense(64, input_shape=(X_train.shape[1],), activation='tanh'),
    keras.layers.Dense(64, activation='tanh'),
    keras.layers.Dense(64, activation='tanh'),
    keras.layers.Dense(1, activation='sigmoid'),
    #keras.layers.Dense(1, activation='softmax')
])

print("Model Summary", model.summary())

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ normalization_14                │ (2649, 10)             │            21 │
│ (Normalization)                 │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_64 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_65 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_66 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_67 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 21 (88.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 21 (88.00 B)

Model Summary None


In [141]:
print(X_train)
print(y_train)
model.compile(optimizer=keras.optimizers.Adam(learning_rate=1e-3), loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), metrics=['accuracy'])
model.fit(X_train, y_train, epochs=200, batch_size=64)

tf.Tensor(
[[ 0.910887    0.25158188  0.64350116 ... -1.0492979   1.1943544
  -1.0285858 ]
 [ 0.3597359  -0.16668607  0.64350116 ... -0.2104043   0.631593
   0.1461535 ]
 [-1.8448685   0.39100453  1.2178389  ...  1.8029402   1.1943544
   2.1599925 ]
 ...
 [ 1.4620382  -1.1426446   1.2178389  ... -0.54596174  0.631593
  -0.18948631]
 [ 0.910887   -0.86379933  0.64350116 ... -0.71374047  0.631593
  -0.692946  ]
 [-0.1914152   0.11215922 -0.5051744  ...  0.4607106  -0.49392986
   0.4817933 ]], shape=(2649, 10), dtype=float32)
[[1]
 [1]
 [0]
 ...
 [1]
 [0]
 [1]]
Epoch 1/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5948 - loss: 0.6663
Epoch 2/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6538 - loss: 0.6098 
Epoch 3/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6478 - loss: 0.5939 
Epoch 4/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6513 - loss: 0.5911 
Epoch 5/200
42/42 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6485 - loss: 0.5738 
Epoch 

In [143]:
#save the model
model.save("src/models/model_with_frets_and_strings.h5")